In [5]:
import sqlite3
import pandas as pd

import pystac_client
import odc.stac

import os

In [6]:
# Check if the operating system is Linux
if os.name == 'posix':
    db_path = '/data/oa3802fa25/GlobalMeltdown/fire.db'
else:
    db_path = 'fire.db'

# Connect to the appropriate database
conn = sqlite3.connect(db_path)

# Create a cursor object
cursor = conn.cursor()

In [ ]:
# set this to True if you run it the first time, to create the database
first = False

if first == True:
    # Read CSV file into a pandas DataFrame
    df = pd.read_csv('./Data/base_data_1986-2018.csv')

    df.drop('time', axis=1, inplace=True)
    df.drop('index_right', axis=1, inplace=True)
    df.month = pd.to_datetime(df.month, format='%Y-%m')
    df = df[df['month'].dt.year != 1986]

    # Write the DataFrame to the SQLite database
    df.to_sql('ground_data', conn, if_exists='replace', index=False)

    # Commit the changes
    conn.commit()

In [8]:
df

,month,lat,lon,geometry,mrso,sfcWind,tas,hurs,pdsi,MEI,SOI,bushfire_count,prescribed_count,grid_id
107004,1987-01-01,-39.05,146.34,POINT (146.34 -39.05),619.038818,6.105469,288.106934,73.959961,0.486829,1.03,-0.75,0.0,0.0,5831.0
107005,1987-01-01,-38.94,146.34,POINT (146.34 -38.94),624.335938,5.265625,288.486084,72.843628,0.486829,1.03,-0.75,0.0,4.0,5832.0
107006,1987-01-01,-38.94,146.45,POINT (146.45 -38.94),612.244385,5.552734,288.525879,72.522461,0.486829,1.03,-0.75,0.0,0.0,5942.0
107007,1987-01-01,-38.83,145.90,POINT (145.9 -38.83),930.560791,6.072266,288.916260,71.438843,2.082829,1.03,-0.75,0.0,0.0,5393.0
107008,1987-01-01,-38.83,146.01,POINT (146.01 -38.83),937.969727,5.759766,288.786865,71.835022,2.082829,1.03,-0.75,0.0,0.0,5503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294121,1988-09-01,-28.27,150.63,POINT (150.63 -28.27),951.135254,4.023438,290.682617,61.785828,4.627917,-1.80,1.92,0.0,0.0,10668.0
294122,1988-09-01,-28.27,150.74,POINT (150.74 -28.27),828.647217,3.914062,290.753174,61.535645,4.627917,-1.80,1.92,0.0,0.0,10778.0
294123,1988-09-01,-28.27,150.85,POINT (150.85 -28.27),964.261719,3.744141,290.259521,63.565063,4.627917,-1.80,1.92,0.0,0.0,10888.0
294124,1988-09-01,-28.27,150.96,POINT (150.96 -28.27),935.252686,3.628906,290.046631,64.352417,4.627917,-1.80,1.92,0.0,0.0,10998.0


In [ ]:
# fetch data from fire.db
cursor.execute('''SELECT * from ground_data LIMIT 10''')
cursor.fetchall()

In [9]:
#Get all tables
def get_all_tables(cursor):
    sql_query = """SELECT name FROM sqlite_master  
      WHERE type='table';"""
    cursor.execute(sql_query)
    return cursor.fetchall()

# Get column names and data types for each table
def get_table_info(cursor, table_name):
    cursor.execute(f'PRAGMA table_info({table_name})')
    return cursor.fetchall()

In [10]:
tables = [table[0] for table in get_all_tables(cursor)]

# printing all tables and datatypes of fire.db
for table in tables:
    print(f"Table: {table}")
    for column in get_table_info(cursor, table):
        print(f"Column: {column[1]}, Type: {column[2]}")
    print("\n")

Table: ground_data
Column: month, Type: TIMESTAMP
Column: lat, Type: REAL
Column: lon, Type: REAL
Column: geometry, Type: TEXT
Column: mrso, Type: REAL
Column: sfcWind, Type: REAL
Column: tas, Type: REAL
Column: hurs, Type: REAL
Column: pdsi, Type: REAL
Column: MEI, Type: REAL
Column: SOI, Type: REAL
Column: bushfire_count, Type: REAL
Column: prescribed_count, Type: REAL
Column: grid_id, Type: REAL




In [11]:
# loading catalog from database
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

In [12]:
# configure the AWS connection
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

In [ ]:
# function to get month, lat, long and grid id from database
def get_month_lat_long(cursor):
    # define how many entries you want or all (remove LIMIT 10)
    sql_query = '''SELECT month, lat, lon, grid_id from ground_data LIMIT 10'''
    cursor.execute(sql_query)
    return cursor.fetchall()

# def create_stac_data_table(cursor, conn):
#     # Create a new table with the required columns
#     cursor.execute('''
#     CREATE TABLE IF NOT EXISTS stac_data (
#         time TEXT,
#         y REAL,
#         x REAL,
#         grid_id REAL,
#         bs_pc_10 REAL,
#         bs_pc_50 REAL,
#         bs_pc_90 REAL,
#         pv_pc_10 REAL,
#         pv_pc_50 REAL,
#         pv_pc_90 REAL,
#         npv_pc_10 REAL,
#         npv_pc_50 REAL,
#         npv_pc_90 REAL
#     )
#     ''')

#     # Commit the changes
#     conn.commit()

# def write_stac_data_to_db(cursor, conn, data):
#     for item in data:
#         cursor.execute('''
#         INSERT INTO stac_data (time, y, x, grid_id, bs_pc_10, bs_pc_50, bs_pc_90, pv_pc_10, pv_pc_50, pv_pc_90, npv_pc_10, npv_pc_50, npv_pc_90)
#         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
#         ''', (
#             item['time'], item['y'], item['x'], item['grid_id'],
#             item['bs_pc_10'], item['bs_pc_50'], item['bs_pc_90'],
#             item['pv_pc_10'], item['pv_pc_50'], item['pv_pc_90'],
#             item['npv_pc_10'], item['npv_pc_50'], item['npv_pc_90']
#         ))

#     # Commit the changes
#     conn.commit()

In [20]:
#create_stac_data_table(cursor, conn)

In [21]:
# data_tiles = get_month_lat_long(cursor)

# print(len(data_tiles))

# for data in data_tiles:

#     bbox = [data[2] - 0.05, data[1] - 0.05, data[2] + 0.05, data[1] + 0.05]

#     start_date = f"{data[0][:4]}-01-01"
#     end_date = f"{data[0][:4]}-12-31"

#     print(f"Searching for data in {start_date} to {end_date} for lat: {data[1]} and long: {data[2]}")

#     # Set product ID as the STAC "collection"
#     collections = ["ga_ls_fc_pc_cyear_3"]

#     # Build a query with the parameters above
#     query = catalog.search(
#         bbox=bbox,
#         collections=collections,
#         datetime=f"{start_date}/{end_date}",
#     )

#     if len(list(query.items())) == 0:
#         print("No data found")
#         continue

#     # Search the STAC catalog for all items matching the query
#     items = list(query.items())
#     print(f"Found: {len(items):d} datasets")

#     # Load the data using the odc.stac module
#     ds = odc.stac.load(
#                 items=items,
#                 crs="EPSG:3577",
#                 lat=(bbox[1], bbox[3]),
#                 lon=(bbox[0], bbox[2]),
#                 time=(start_date, end_date))
    
#     data_entry = {
#         'time': data[0],
#         'grid_id': data[3],
#         'bs_pc_10': float(ds.bs_pc_10.mean().values),
#         'bs_pc_50': float(ds.bs_pc_50.mean().values),
#         'bs_pc_90': float(ds.bs_pc_90.mean().values),
#         'pv_pc_10': float(ds.pv_pc_10.mean().values),
#         'pv_pc_50': float(ds.pv_pc_50.mean().values),
#         'pv_pc_90': float(ds.pv_pc_90.mean().values),
#         'npv_pc_10': float(ds.npv_pc_10.mean().values),
#         'npv_pc_50': float(ds.npv_pc_50.mean().values),
#         'npv_pc_90': float(ds.npv_pc_90.mean().values)
#     }

#     write_stac_data_to_db(cursor, conn, [data_entry])


In [22]:
# cursor.execute('''SELECT * FROM stac_data''')
# rows = cursor.fetchall()

# for row in rows:
#     print(row)

In [17]:
# Function to check if a column exists in a table
def column_exists(cursor, table_name, column_name):
	cursor.execute(f"PRAGMA table_info({table_name})")
	columns = [info[1] for info in cursor.fetchall()]
	return column_name in columns

# List of columns to add
columns_to_add = [
	'bs_pc_10 REAL DEFAULT NAN',
	'bs_pc_50 REAL DEFAULT NAN',
	'bs_pc_90 REAL DEFAULT NAN',
	'pv_pc_10 REAL DEFAULT NAN',
	'pv_pc_50 REAL DEFAULT NAN',
	'pv_pc_90 REAL DEFAULT NAN',
	'npv_pc_10 REAL DEFAULT NAN',
	'npv_pc_50 REAL DEFAULT NAN',
	'npv_pc_90 REAL DEFAULT NAN'
]

# Alter the table to add new columns if they don't already exist
for column in columns_to_add:
	column_name = column.split()[0]
	if not column_exists(cursor, 'ground_data', column_name):
		cursor.execute(f'ALTER TABLE ground_data ADD COLUMN {column}')

# Commit the changes
conn.commit()

In [18]:
# function to update each row in the data base table
def update_ground_data(cursor, conn, data):
    cursor.execute('''
    UPDATE ground_data
    SET bs_pc_10 = ?, bs_pc_50 = ?, bs_pc_90 = ?, pv_pc_10 = ?, pv_pc_50 = ?, pv_pc_90 = ?, npv_pc_10 = ?, npv_pc_50 = ?, npv_pc_90 = ?
    WHERE grid_id = ? AND month = ?
    ''', (
        data['bs_pc_10'], data['bs_pc_50'], data['bs_pc_90'],
        data['pv_pc_10'], data['pv_pc_50'], data['pv_pc_90'],
        data['npv_pc_10'], data['npv_pc_50'], data['npv_pc_90'],
        data['grid_id'], data['time']
    ))

    # Commit the changes
    conn.commit()

In [ ]:
# get information from database
data_tiles = get_month_lat_long(cursor)

# print length of data
print(len(data_tiles))

# for each dataset, get the data from AWS
for data in data_tiles:

    bbox = [data[2] - 0.05, data[1] - 0.05, data[2] + 0.05, data[1] + 0.05]

    start_date = f"{data[0][:4]}-01-01"
    end_date = f"{data[0][:4]}-12-31"

    print(f"Searching for data in {start_date} to {end_date} for lat: {data[1]} and long: {data[2]}")

    # Set product ID as the STAC "collection"
    collections = ["ga_ls_fc_pc_cyear_3"]

    try:
        # Build a query with the parameters above
        query = catalog.search(
            bbox=bbox,
            collections=collections,
            datetime=f"{start_date}/{end_date}",
        )

        if len(list(query.items())) == 0:
            print("No data found")
            continue

    except Exception as e:
        
        print(f"An error occurred: {e}")
        continue

    # Search the STAC catalog for all items matching the query
    items = list(query.items())
    print(f"Found: {len(items):d} datasets")

    # Load the data using the odc.stac module
    ds = odc.stac.load(
                items=items,
                crs="EPSG:3577",
                lat=(bbox[1], bbox[3]),
                lon=(bbox[0], bbox[2]),
                time=(start_date, end_date))
    
    data_entry = {
        'time': data[0],
        'grid_id': data[3],
        'bs_pc_10': float(ds.bs_pc_10.mean().values),
        'bs_pc_50': float(ds.bs_pc_50.mean().values),
        'bs_pc_90': float(ds.bs_pc_90.mean().values),
        'pv_pc_10': float(ds.pv_pc_10.mean().values),
        'pv_pc_50': float(ds.pv_pc_50.mean().values),
        'pv_pc_90': float(ds.pv_pc_90.mean().values),
        'npv_pc_10': float(ds.npv_pc_10.mean().values),
        'npv_pc_50': float(ds.npv_pc_50.mean().values),
        'npv_pc_90': float(ds.npv_pc_90.mean().values)
    }

    update_ground_data(cursor, conn, data_entry)

In [ ]:
# checking data in table
cursor.execute('''SELECT * from ground_data''')
cursor.fetchall()

In [ ]:
# checking length of database
cursor.execute('SELECT COUNT(*) FROM ground_data')
count = cursor.fetchone()[0]
print(f"Number of entries in ground_data table: {count}")

In [ ]:
# def delete_all_tables(cursor, conn):
#     tables = get_all_tables(cursor)
#     for table in tables:
#         cursor.execute(f"DROP TABLE IF EXISTS {table[0]}")
#     conn.commit()

# delete_all_tables(cursor, conn)